In [1]:
import sys
sys.path.append("..")

import numpy as np
from propagation_simulator.environment import environment
from classes.nfpo import NFPO
from matplotlib import pyplot as plt

## 環境設定

In [2]:
frequency = 920.4e6
c = 2.998e8
wave_length = c/frequency
sample_count = 100
tee = False

In [3]:
tag_location= [1,3]
antennas_population = 4
k=-2
antennas_space=0.1

## 位相サンプル生成

In [4]:
env = environment(
    antennas_space=antennas_space,
    antennas_population=antennas_population,
    tag=tag_location,
    frequency=frequency,
    k=k,
    sample_count=sample_count,
)
antennas = env.antennas
vectors = env.vectors
actual_phases, phases, phases_with_noise = env.get_phases()
actual_n, n, n_with_noise = env.get_n()

## 最適化モデル

In [5]:
nfpo = NFPO(
    sample_count=sample_count,
    antennas_space=antennas_space,
    phases=phases,
    antennas_population=antennas_population,
    wave_length = wave_length,
    tag_location = tag_location,
    initial_value = np.array([0,2.5]),
    y_err_range = 1.5,
    x_err_range = 2,
    k=k,
    is_noise_included=True,
)

### 本推定

In [6]:
x_grid_list = np.arange(-2.0,2.0,1.0) + 0.5
y_grid_list = np.arange(1.0,4.0,1.0) + 0.5

In [7]:
best_initial_value = np.array([0,2])
min_obj = 1e10
solve_time = 0
error = 0
for x in x_grid_list:
    for y in y_grid_list:
        initial_value = np.array([x,y])
        nfpo = NFPO(
            sample_count=sample_count,
            antennas_space=antennas_space,
            phases=phases,
            antennas_population=antennas_population,
            wave_length = wave_length,
            tag_location = tag_location,
            initial_value = initial_value,
            y_err_range = 0.6,
            x_err_range = 0.6,
            k=k,
            is_noise_included=True,
        )
        result = nfpo.solve(tee=tee, time_limit=60)
        print(f"initial_value = {initial_value}")
        print(f"    obj_value = {result["values"]["obj"]}")
        print(f"    solve time = {result["solving_time"]}[s]")
        print(f"    error = {result["values"]["error"]}[m]")
        if min_obj > result["values"]["obj"]:
            solve_time = result["solving_time"]
            min_obj = result["values"]["obj"]
            error = result["values"]["error"]
            best_initial_value = np.array([result["values"]["x"], result["values"]["y"]])

initial_value = [-1.5  1.5]
    obj_value = 0.40810063843425143
    solve time = 0.32[s]
    error = 2.831960452754589[m]
initial_value = [-1.5  2.5]
    obj_value = 0.5441134731245766
    solve time = 0.35[s]
    error = 2.1876027599023353[m]
initial_value = [-1.5  3.5]
    obj_value = 0.7390697442183697
    solve time = 0.39[s]
    error = 1.9001592288450557[m]
initial_value = [-0.5  1.5]
    obj_value = 0.02057200932013766
    solve time = 0.32[s]
    error = 2.1646640419118235[m]
initial_value = [-0.5  2.5]
    obj_value = 0.0696969717253635
    solve time = 0.33[s]
    error = 1.3425071240034145[m]
initial_value = [-0.5  3.5]
    obj_value = 0.15038258762993978
    solve time = 0.25[s]
    error = 0.900256132027479[m]
initial_value = [0.5 1.5]
    obj_value = 3.217347878885887e-05
    solve time = 1.33[s]
    error = 0.976595902257045[m]
initial_value = [0.5 2.5]
    obj_value = 1.6051665643621478e-09
    solve time = 2.21[s]
    error = 0.015744901387973572[m]
initial_value = [0.

In [8]:
print(solve_time)
print(min_obj)
print(error)
print(best_initial_value)

2.21
1.6051665643621478e-09
0.015744901387973572
[0.98838927 3.01063451]


In [9]:
nfpo = NFPO(
        sample_count=sample_count,
        antennas_space=antennas_space,
        phases=phases_with_noise,
        antennas_population=antennas_population,
        wave_length = wave_length,
        tag_location = tag_location,
        initial_value = best_initial_value,
        y_err_range = 0.1,
        x_err_range = 0.1,
        k=k,
        is_noise_included=True,
    )
result = nfpo.solve(tee=tee, time_limit=60)
for key,value in result.items():
    if key != "values":
        print(key + ": " + str(value))
    else:
        print("coordinate: " + str([value["x"], value["y"]]))
        print("N: " + str(value["N"]))
        print("obj_value: " + str(value["obj"]))
        print("angle: " + str(value["angle"]))
        print("error: " + str(value["error"]))
        print("error_vector: ", str(value["error_vector"]))

solving_time: 0.54
coordinate: [0.9998113579002357, 3.00029128597292]
N: [-20. -20. -20. -19.]
obj_value: 9.24998401265662e-06
angle: [0.78530383 0.76835478 0.75083169 0.73271623]
error: 0.00034703509883489517
error_vector:  [-0.00018864  0.00029129]
